## オブザーバビリティを高める機能

必要なライブラリをダウンロードします。

In [ ]:
%pip install -r ../requirements.txt

実行に必要な環境変数を読み込みます。

In [1]:
import os, warnings, time, uuid

# おまじない
warnings.filterwarnings("ignore")

endpoint = "http://localhost:3000"
public_key = os.getenv("PUBLIC_KEY")
secret_key = os.getenv("SECRET_KEY")

### セッション

各トレースを同一グループにするために、セッションIDをUUID v4ベースで生成する

In [2]:
import uuid

session_id = str(uuid.uuid4())

`@observe()`デコレーターを使用した場合

In [6]:
from langfuse.decorators import langfuse_context, observe

langfuse_context.configure(
    public_key=public_key,
    secret_key=secret_key,
    host=endpoint
)

@observe()
def search_the_bigbaby():
    # 同期の検索処理を想定
    time.sleep(2)
    langfuse_context.update_current_trace(
        session_id=session_id
    )

search_the_bigbaby()
search_the_bigbaby()

Low-Level SDKを使用した場合

In [7]:
from langfuse import Langfuse

langfuse = Langfuse(
    public_key=public_key,
    secret_key=secret_key,
    host=endpoint
)

trace = langfuse.trace(
    name="Ask the BigBaBy",
    session_id=session_id
)

span = trace.span(
    name="Search the BigBaBy's dictionary"
)
# 同期の検索処理を想定
time.sleep(2)
span.end()

LangChainのCallback実装を使った場合

In [9]:
from langfuse.callback import CallbackHandler

callback_handler = CallbackHandler(
    public_key=public_key,
    secret_key=secret_key,
    host=endpoint,
    session_id=session_id
)

if os.getenv("COHERE_API_KEY") == None:
    from langchain_openai.chat_models import ChatOpenAI
    openai_api_key = os.getenv("OPENAI_API_KEY")
    model = "gpt-4o-mini"
    chat = ChatOpenAI(api_key=openai_api_key, model=model)
else:
    from langchain_cohere.chat_models import ChatCohere
    cohere_api_key = os.getenv("COHERE_API_KEY")
    model = "command-r-plus"
    chat = ChatCohere(cohere_api_key=cohere_api_key, model=model)

chat.invoke(
    input="肉料理のおすすめ教えて",
    config={"callbacks": [callback_handler]}
)
chat.invoke(
    input="肉料理のおすすめ教えて",
    config={"callbacks": [callback_handler]}
)

validate_env
generate
{'message': '肉料理のおすすめ教えて', 'chat_history': [], 'model': 'command-r-plus'}
basecohere.chat
text='喜んでいただけるか分かりませんが、いくつかおすすめの肉料理を紹介します。\n\n- ローストビーフ： 牛肉の塊をオーブンで焼いた料理で、柔らかくジューシーな食感と深い味わいが楽しめます。\n- 豚の角煮： 豚バラ肉を醤油や砂糖などで長時間煮込んだ料理で、とろけるような食感と甘辛い味付けがご飯に良く合います。\n- チキンティッカ： インド料理のグリルチキンで、ヨーグルトと香辛料に漬け込んだ鶏肉を串に刺して焼いたものです。香り高くジューシーな味わいが特徴です。\n- ラムチョップ： 仔羊の骨付きロース肉をグリルしたもので、柔らかくクセのない赤身肉が楽しめます。\n- 牛タン焼き： 牛タンをスライスして焼いた料理で、柔らかくて旨味たっぷりの味わいが魅力です。\n\n以上、お好みや気分に合わせてお楽しみください。' generation_id='99632c08-b0d9-4778-a9ca-c59a6e5fe1ec' citations=None documents=None is_search_required=None search_queries=None search_results=None finish_reason='COMPLETE' tool_calls=None chat_history=[Message_User(message='肉料理のおすすめ教えて', tool_calls=None, role='USER'), Message_Chatbot(message='喜んでいただけるか分かりませんが、いくつかおすすめの肉料理を紹介します。\n\n- ローストビーフ： 牛肉の塊をオーブンで焼いた料理で、柔らかくジューシーな食感と深い味わいが楽しめます。\n- 豚の角煮： 豚バラ肉を醤油や砂糖などで長時間煮込んだ料理で、とろけるような食感と甘辛い味付けがご飯に良く合います。\n- チキンティッカ： インド料理のグリルチキンで、ヨーグルトと香辛料に漬け込んだ鶏肉を串に刺して焼いたものです。香り高くジューシーな味わいが特徴

AIMessage(content='喜んでいただけるか分かりませんが、いくつかおすすめの肉料理を紹介します。\n\n- ローストビーフ： 牛肉の塊をオーブンでゆっくりと焼き上げた料理です。ジューシーで柔らかく、赤身の美味しさを存分に楽しめます。\n- 豚の角煮： 豚バラ肉を醤油や砂糖などで長時間煮込んだ料理です。脂が柔らかく、とろけるような食感が魅力です。\n- チキンティッカ： インド料理のグリルチキンで、ヨーグルトと香辛料に漬け込んだ鶏肉を串に刺して焼き上げます。香り高く、ジューシーな味わいです。\n- ラムチョップ： ラム肉のリブ（肋骨）部分をカットしたもので、骨付きのままグリルしたり、オーブンで焼いたりして楽しめます。独特の香りと旨味があります。\n- 牛タン焼き： 牛タンをスライスして焼き上げた料理で、仙台の名物としても知られています。柔らかく、噛むほどに旨味が広がります。\n\nこれらの料理は、家庭でも比較的簡単に作れるものから、少し手間のかかるものまで様々ですが、ぜひお試しください。', additional_kwargs={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': 'cbbc5721-d4b8-4f16-9d1a-4241031955c5', 'token_count': {'input_tokens': 206.0, 'output_tokens': 294.0}}, response_metadata={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': 'cbbc5721-d4b8-4f16-9d1a-4241031955c5', 'token_count': {'input_tokens': 206.0, 'output_tokens': 294.0}}, id='run-5b1ff9af-87c0-4729-b84e

### ユーザー

`@observe()`デコレーターを使用した場合

In [10]:
from langfuse.decorators import langfuse_context, observe

langfuse_context.configure(
    public_key=public_key,
    secret_key=secret_key,
    host=endpoint
)

@observe()
def search_the_bigbaby():
    # 同期の検索処理を想定
    time.sleep(2)
    langfuse_context.update_current_trace(
        user_id="user@example.com"
    )

search_the_bigbaby()

Low-Level SDKを使用した場合

In [11]:
from langfuse import Langfuse

langfuse = Langfuse(
    public_key=public_key,
    secret_key=secret_key,
    host=endpoint
)

trace = langfuse.trace(
    name="Ask the BigBaBy",
    user_id="user@example.com"
)

span = trace.span(
    name="Search the BigBaBy's dictionary"
)
# 同期の検索処理を想定
time.sleep(2)
span.end()

LangChainのCallback実装を使った場合

In [12]:
from langfuse.callback import CallbackHandler

callback_handler = CallbackHandler(
    public_key=public_key,
    secret_key=secret_key,
    host=endpoint,
    user_id="user@example.com"
)

if os.getenv("COHERE_API_KEY") == None:
    from langchain_openai.chat_models import ChatOpenAI
    openai_api_key = os.getenv("OPENAI_API_KEY")
    model = "gpt-4o-mini"
    chat = ChatOpenAI(api_key=openai_api_key, model=model)
else:
    from langchain_cohere.chat_models import ChatCohere
    cohere_api_key = os.getenv("COHERE_API_KEY")
    model = "command-r-plus"
    chat = ChatCohere(cohere_api_key=cohere_api_key, model=model)

chat.invoke(
    input="肉料理のおすすめ教えて",
    config={"callbacks": [callback_handler]}
)

validate_env
generate
{'message': '肉料理のおすすめ教えて', 'chat_history': [], 'model': 'command-r-plus'}
basecohere.chat
text='喜んでいただけるか分かりませんが、いくつかおすすめの肉料理を紹介します。\n\n- ローストビーフ： 牛肉の塊をオーブンなどでゆっくりと焼き上げた料理です。ジューシーでやわらかな食感と、深い味わいが楽しめます。\n- 豚の角煮： 豚バラ肉を醤油や砂糖などで甘辛く煮込んだ料理です。とろけるような食感と、濃厚な味わいが魅力です。\n- チキンティッカ： インド料理の代表的な肉料理で、ヨーグルトと香辛料に漬け込んだ鶏肉を串に刺して焼いたものです。スパイシーでジューシーな味わいが楽しめます。\n- ラムチョップ： ラム肉のロース部分を骨付きのまま焼いた料理です。やわらかな食感と、独特の香りと風味が魅力です。\n- 牛タン焼き： 牛タンをスライスして焼いた料理です。やわらかな食感と、独特の旨みが楽しめます。\n\n以上、お口に合うものがあれば幸いです。' generation_id='86439133-24a5-4b93-a14a-7890004e8ac6' citations=None documents=None is_search_required=None search_queries=None search_results=None finish_reason='COMPLETE' tool_calls=None chat_history=[Message_User(message='肉料理のおすすめ教えて', tool_calls=None, role='USER'), Message_Chatbot(message='喜んでいただけるか分かりませんが、いくつかおすすめの肉料理を紹介します。\n\n- ローストビーフ： 牛肉の塊をオーブンなどでゆっくりと焼き上げた料理です。ジューシーでやわらかな食感と、深い味わいが楽しめます。\n- 豚の角煮： 豚バラ肉を醤油や砂糖などで甘辛く煮込んだ料理です。とろけるような食感と、濃厚な味わいが魅力です。\n- チキンティッカ： インド料理の代表的な肉料理で、ヨーグルトと香辛料に漬け込んだ鶏肉を串

AIMessage(content='喜んでいただけるか分かりませんが、いくつかおすすめの肉料理を紹介します。\n\n- ローストビーフ： 牛肉の塊をオーブンなどでゆっくりと焼き上げた料理です。ジューシーでやわらかな食感と、深い味わいが楽しめます。\n- 豚の角煮： 豚バラ肉を醤油や砂糖などで甘辛く煮込んだ料理です。とろけるような食感と、濃厚な味わいが魅力です。\n- チキンティッカ： インド料理の代表的な肉料理で、ヨーグルトと香辛料に漬け込んだ鶏肉を串に刺して焼いたものです。スパイシーでジューシーな味わいが楽しめます。\n- ラムチョップ： ラム肉のロース部分を骨付きのまま焼いた料理です。やわらかな食感と、独特の香りと風味が魅力です。\n- 牛タン焼き： 牛タンをスライスして焼いた料理です。やわらかな食感と、独特の旨みが楽しめます。\n\n以上、お口に合うものがあれば幸いです。', additional_kwargs={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '86439133-24a5-4b93-a14a-7890004e8ac6', 'token_count': {'input_tokens': 206.0, 'output_tokens': 258.0}}, response_metadata={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '86439133-24a5-4b93-a14a-7890004e8ac6', 'token_count': {'input_tokens': 206.0, 'output_tokens': 258.0}}, id='run-51bee2bb-7493-4abd-9b23-65f3679d45a2-0', usage_metadata={'input_tokens': 206, 'outpu

### メタデータ

`@observe()`デコレーターを使用した場合

In [13]:
from langfuse.decorators import langfuse_context, observe

langfuse_context.configure(
    public_key=public_key,
    secret_key=secret_key,
    host=endpoint
)

@observe()
def search_the_bigbaby():
    # 同期の検索処理を想定
    time.sleep(2)
    langfuse_context.update_current_trace(
        metadata={
            "search_params_top_k": 3
        }
    )

search_the_bigbaby()

Low-Level SDKを使用した場合

In [11]:
from langfuse import Langfuse

langfuse = Langfuse(
    public_key=public_key,
    secret_key=secret_key,
    host=endpoint
)

trace = langfuse.trace(
    name="Ask the BigBaBy",
    metadata={
        "search_params_top_k": 3
    }
)

span = trace.span(
    name="Search the BigBaBy's dictionary"
)
# 同期の検索処理を想定
time.sleep(2)
span.end()

LangChainのCallback実装を使った場合

In [12]:
from langfuse.callback import CallbackHandler

callback_handler = CallbackHandler(
    public_key=public_key,
    secret_key=secret_key,
    host=endpoint,
    metadata={
        "search_params_top_k": 3
    }
)

if os.getenv("COHERE_API_KEY") == None:
    from langchain_openai.chat_models import ChatOpenAI
    openai_api_key = os.getenv("OPENAI_API_KEY")
    model = "gpt-4o-mini"
    chat = ChatOpenAI(api_key=openai_api_key, model=model)
else:
    from langchain_cohere.chat_models import ChatCohere
    cohere_api_key = os.getenv("COHERE_API_KEY")
    model = "command-r-plus"
    chat = ChatCohere(cohere_api_key=cohere_api_key, model=model)

chat.invoke(
    input="肉料理のおすすめ教えて",
    config={"callbacks": [callback_handler]}
)

AIMessage(content='喜んで！肉料理のおすすめをいくつか紹介しますね。\n\n- ローストビーフ： 牛肉の塊を低温でじっくりと焼き上げた料理です。柔らかくジューシーな食感と、赤身の美味しさを楽しめます。\n- 豚の角煮： 豚バラ肉を醤油や砂糖などで甘辛く煮込んだ料理です。柔らかくホロホロとした食感と、濃厚な味わいが魅力です。\n- チキンのトマト煮込み： 鶏もも肉をトマト缶や野菜と一緒に煮込んだ料理です。トマトの酸味と鶏肉の旨味が絡み合い、さっぱりとした味わいです。\n- ラムチョップのグリル： ラム肉の骨付きロース肉をグリルした料理です。香草やスパイスでマリネして焼くことで、独特の香りとジューシーな味わいを楽しめます。\n- 牛タン焼き： 牛タンを薄切りにして焼き上げた料理です。柔らかくて旨味たっぷりで、ビールや日本酒などのお酒にもよく合います。\n\n以上、肉料理のおすすめでした。ぜひお試しください！', additional_kwargs={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': 'b0990de7-a09b-4c59-b932-21ee013abbba', 'token_count': {'input_tokens': 206.0, 'output_tokens': 290.0}}, response_metadata={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': 'b0990de7-a09b-4c59-b932-21ee013abbba', 'token_count': {'input_tokens': 206.0, 'output_tokens': 290.0}}, id='run-d121b12a-6815-4f14-bd37-f282f358d442-0', usage_metadata={'input

### タグ

`@observe()`デコレーターを使用した場合

In [13]:
from langfuse.decorators import langfuse_context, observe

langfuse_context.configure(
    public_key=public_key,
    secret_key=secret_key,
    host=endpoint
)

@observe()
def search_the_bigbaby():
    # 同期の検索処理を想定
    time.sleep(2)
    langfuse_context.update_current_trace(
        tags=["production", "summarize"]
    )

search_the_bigbaby()

Low-Level SDKを使った場合

In [14]:
from langfuse import Langfuse

langfuse = Langfuse(
    public_key=public_key,
    secret_key=secret_key,
    host=endpoint
)

trace = langfuse.trace(
    name="Ask the BigBaBy",
    tags=["production", "summarize"]
)

span = trace.span(
    name="Search the BigBaBy's dictionary"
)
# 同期の検索処理を想定
time.sleep(2)
span.end()

LangChainのCallback実装を使った場合

In [15]:
from langfuse.callback import CallbackHandler

callback_handler = CallbackHandler(
    public_key=public_key,
    secret_key=secret_key,
    host=endpoint,
    tags=["production", "summarize"]
)

if os.getenv("COHERE_API_KEY") == None:
    from langchain_openai.chat_models import ChatOpenAI
    openai_api_key = os.getenv("OPENAI_API_KEY")
    model = "gpt-4o-mini"
    chat = ChatOpenAI(api_key=openai_api_key, model=model)
else:
    from langchain_cohere.chat_models import ChatCohere
    cohere_api_key = os.getenv("COHERE_API_KEY")
    model = "command-r-plus"
    chat = ChatCohere(cohere_api_key=cohere_api_key, model=model)

chat.invoke(
    input="肉料理のおすすめ教えて",
    config={"callbacks": [callback_handler]}
)

AIMessage(content='喜んでいただけるか分かりませんが、いくつかおすすめの肉料理を紹介します。\n\n- ローストビーフ： 牛肉の塊をオーブンなどでゆっくりと焼き上げた料理です。ジューシーでやわらかな食感と、深い味わいが楽しめます。\n- 豚の角煮： 豚バラ肉を醤油や砂糖などで長時間煮込んだ料理です。とろけるような食感と、甘辛い味付けがご飯によく合います。\n- チキンティッカ： インド料理のグリルチキンです。ヨーグルトや香辛料に漬け込んだ鶏肉を串に刺して焼き上げます。香り高く、ジューシーな味わいが特徴です。\n- ラムチョップ： 子羊のリブ（あばら骨）部分をグリルしたり、オーブンで焼いたりした料理です。やわらかな食感と、独特の香りが楽しめます。\n- 牛タン焼き： 牛タンをスライスして焼き上げた料理です。やわらかな食感と、独特の風味が人気です。\n\n以上、お口に合うものがあれば幸いです。', additional_kwargs={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '486a822b-8bf0-4087-94eb-8f17c70020b8', 'token_count': {'input_tokens': 206.0, 'output_tokens': 270.0}}, response_metadata={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '486a822b-8bf0-4087-94eb-8f17c70020b8', 'token_count': {'input_tokens': 206.0, 'output_tokens': 270.0}}, id='run-e361d9af-0903-4a0d-ac6b-ffc802bcf855-0', usage_metadata={'input_tokens